In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# isort: off
# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)

import os
import sys


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()
# third party
from email_helpers import get_email_server
from email_helpers import load_users
from job_helpers import load_jobs
from job_helpers import save_jobs
# isort: on

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

# Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
email_server, smtp_server = get_email_server()

# Download results

In [ ]:
users = load_users(high_client)
jobs = load_jobs(users, high_client)

In [ ]:
# submitted_jobs = [job for job in jobs if job.is_submitted]
reviewed_jobs = [job for job in jobs if job.admin_reviewed]
reviewed_jobs_should_succeed = [j for j in reviewed_jobs if j.should_succeed]
reviewed_jobs_should_fail = [j for j in reviewed_jobs if not j.should_succeed]

print(
    f"{len(reviewed_jobs)=}, {len(reviewed_jobs_should_succeed)=}, {len(reviewed_jobs_should_fail)=}"
)

In [ ]:
# TODO: test jobs that were never approved
# they seem to give weird errors like
# "You uploaded an ActionObject that is not yet in the blob storage"

In [ ]:
for job in reviewed_jobs_should_succeed:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method
    j = api_method(blocking=False)
    res = j.wait()

    if isinstance(res, sy.SyftError):
        raise sy.SyftException(public_message="Expected success, got error")

    result = res.get()
    job.result_as_expected = True

In [ ]:
for job in reviewed_jobs_should_fail:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    j = api_method(blocking=False)
    res = j.wait()
    if isinstance(res, sy.SyftError):
        job.result_as_expected = True
    else:
        raise sy.SyftException(public_message=f"failed, job didnt raise {type(j)}")

In [ ]:
save_jobs(jobs)

In [ ]:
# TODO fix
expected_jobs = [job for job in jobs if job.result_as_expected]
print(f"got expected_jobs: {len(expected_jobs)} == reviewed_jobs: {len(reviewed_jobs)}")
assert len(reviewed_jobs) == len(expected_jobs)

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()